In [2]:
import numpy as np
import open3d as o3d
import trimesh
import pyrender

from auxiliary import values as v

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
group = 'Gr4'
specimens = v.specimens[group]

Without Landmarks

In [24]:
source_path = v.data_path + f'{group}/3DShape/Tissue/myocardium/2019{specimens[0]}Shape.ply'
target_path = v.data_path + f'{group}/3DShape/Tissue/myocardium/2019{specimens[1]}Shape.ply'

# source_path = v.data_path + f'Gr1/3DShape/Tissue/myocardium/20190208_E2Shape.ply'
# target_path = v.data_path + f'Gr1/3DShape/Membrane/myocardium/20190208_E2_myocardium.ply'

source_mesh = o3d.io.read_triangle_mesh(source_path)
target_mesh = o3d.io.read_triangle_mesh(target_path)

source_pcd = source_mesh.sample_points_uniformly(number_of_points=10000)
target_pcd = target_mesh.sample_points_uniformly(number_of_points=10000)

source_pcd.scale(1 / np.max(source_pcd.get_max_bound() - source_pcd.get_min_bound()), center=source_pcd.get_center())
target_pcd.scale(1 / np.max(target_pcd.get_max_bound() - target_pcd.get_min_bound()), center=target_pcd.get_center())

# Normals estimation
radius_normal = 0.2
source_pcd.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))
target_pcd.estimate_normals(o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

# FPFH features computation
radius_feature = 0.5
fpfh_source = o3d.pipelines.registration.compute_fpfh_feature(source_pcd, o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
fpfh_target = o3d.pipelines.registration.compute_fpfh_feature(target_pcd, o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))

# RANSAC registration
best_fitness = -np.inf
best_transformation = np.eye(4)
num_trials = 20
ransac_n = 4
threshold = 0.1

for i in range(num_trials):
    ransac_result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        target_pcd, source_pcd, fpfh_target, fpfh_source, mutual_filter=False, max_correspondence_distance=threshold,
        estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        ransac_n=ransac_n,
        checkers=[o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
                  o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(threshold)],
        criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(4000000, 500)
    )

    if ransac_result.fitness > best_fitness:
        best_fitness = ransac_result.fitness
        best_transformation = ransac_result.transformation
        
print(best_transformation)
print(best_fitness)

target_pcd.transform(best_transformation)
target_mesh.transform(best_transformation)

o3d.visualization.draw_geometries([source_pcd, target_pcd])
o3d.visualization.draw_geometries([source_mesh, target_mesh])

[[ 9.90157908e-01 -1.35502221e-01 -3.50209206e-02 -1.29102790e+02]
 [ 1.27461516e-01  9.76425398e-01 -1.74203916e-01 -7.74526874e+01]
 [ 5.78003339e-02  1.68025566e-01  9.84086648e-01 -1.15501098e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
0.9274


In [15]:
o3d.visualization.draw_geometries([source_mesh, target_mesh])

With Landmarks